In [ ]:
import sys
import datetime
import os
from environment import Environment # This import will fail as the simulator code cannot be open sourced (yet) :-(
import gym
from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, CallbackList, EvalCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from helper import SaveOnBestTrainingRewardCallback_v2, get_statistics

# Run an experiment

All experiments described in the thesis were conducted by the code below. See the comments of the Variables for more descriptions of the parametrization of the experiments



In [ ]:
start = timer()

NUM_RUNS = 5 # How many different random seeds should be used 
NUM_FORECAST_STEPS = 8 #How many forecast steps to include into the state
ACTION_TYPE = 'continuous' #Choose continuous or discrete action type
USE_PRICE = True # True if Demand Response scenario
BUILDING_MODEL_CLASS = '2R2C'
BUILDING_TYPE = 'i4c' #Define Building type. Caution, types are named different than in the master thesis. (i4c=efficient, 84=old)
RESAMPLE_INTERVAL = '900s' #How many seconds correspond to one time step ?
FORECAST_DOWNSAMPLING_RATE = 1 #Downsampling of the forecast, if a really long forecast is included which would cause a too big state space

# Noise Experiment
SENSOR_NOISE=False, #If set to True, noise will be added to the sensor meassurements
FORECAST_NOISE=False, #If set to True, noise will be added to the forecast 
RANDOMIZE_BUILDING=False, #If set to True, Domain Randomization will be applied during training
DISTURBANCES=False, #If set to True, data from solar irradiation and random opening of windows will be simulated during training


experiment_dir = '' #Path to Experiment

os.makedirs(experiment_dir, exist_ok=True)

for i in range(NUM_RUNS):
    seed = 42 + i

    log_dir = experiment_dir + str(seed) + '/'
    os.makedirs(log_dir, exist_ok=True)

    print("Running experiment and saving outputs to {}".format(log_dir))
    
    # Create environment used for training. Parameters are described above
    train_env = Environment(data_path='',
                            action_type=ACTION_TYPE,
                            resample_interval=RESAMPLE_INTERVAL,
                            evaluation=False, # Set to False during Training
                            num_forecast_steps=NUM_FORECAST_STEPS,
                            heatpump_type='aw', # AW = Air source Water was used in the master thesis
                            seed=seed,
                            sensor_noise=SENSOR_NOISE,
                            forecast_noise=FORECAST_NOISE,
                            randomize_building=RANDOMIZE_BUILDING,
                            disturbances=DISTURBANCES,
                            building_model_class=BUILDING_MODEL_CLASS,
                            forecast_downsampling_rate=FORECAST_DOWNSAMPLING_RATE,
                            building_type=BUILDING_TYPE,
                            use_price=USE_PRICE)

    env = Monitor(train_env, filename=log_dir)
    # DummyVec to be able to use VecNormalize
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    
    # Callback used to obtain best model during training
    save_best_agent_callback = SaveOnBestTrainingRewardCallback_v2(action_type=ACTION_TYPE,
                                                                   check_freq=20000,
                                                                   log_dir=log_dir,
                                                                   resample_interval=RESAMPLE_INTERVAL,
                                                                   building_model_class=BUILDING_MODEL_CLASS,
                                                                   building_type=BUILDING_TYPE,
                                                                   verbose=1,
                                                                   num_forecast_steps=NUM_FORECAST_STEPS,
                                                                   use_price=USE_PRICE,
                                                                   seed=seed)

    # Running PPO using stable baselines    
    model = PPO('MlpPolicy', env, verbose=1, gamma=0.99, learning_rate=0.00005, seed=seed)
    model.learn(total_timesteps=1000000, callback=save_best_agent_callback)
    env.save(log_dir + 'saved_env.pkl')

    del model

    
print ('Elapsed time is %.2fs.' % (timer() - start))

## Generate stats for the experiment

In [ ]:
experiment_dir = '' #Path to Experiment

In [ ]:
   
for directory in os.listdir(experiment_dir):
    run_path = experiment_dir + directory + '/'
    print("getting stats for ", run_path)
    agent = PPO.load(run_path + "best_model.zip")
    
    #get stats on validation data
    stats = get_statistics(agent=agent,
                           action_type=ACTION_TYPE,
                           num_forecast_steps=NUM_FORECAST_STEPS,
                           eval_data_path='',
                           resample_interval=RESAMPLE_INTERVAL,
                           use_price=USE_PRICE,
                           building_model_class=BUILDING_MODEL_CLASS,
                           building_type=BUILDING_TYPE,
                           env_path=run_path + 'saved_env_best.pkl')
    stats.to_csv(run_path + 'eval.csv')
    
    #get stats on test data
    stats = get_statistics(agent=agent,
                           action_type=ACTION_TYPE,
                           num_forecast_steps=NUM_FORECAST_STEPS,
                           eval_data_path='',
                           resample_interval=RESAMPLE_INTERVAL,
                           use_price=USE_PRICE,
                           building_model_class=BUILDING_MODEL_CLASS,
                           building_type=BUILDING_TYPE,
                           env_path=run_path + 'saved_env_best.pkl')
    stats.to_csv(run_path + 'test.csv')
    
    #Get stats directly with noise
    stats = get_statistics(agent=agent,
                           action_type=ACTION_TYPE,
                           num_forecast_steps=NUM_FORECAST_STEPS,
                           eval_data_path='',
                           resample_interval=RESAMPLE_INTERVAL,
                           use_price=USE_PRICE,
                           building_model_class=BUILDING_MODEL_CLASS,
                           building_type=BUILDING_TYPE,
                           sensor_noise=True,
                           forecast_noise=True,
                           env_path=run_path + 'saved_env_best.pkl')
    stats.to_csv(run_path + 'noise-forecast-sensor0501.csv')